In [1]:
import pickle
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn import preprocessing
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
import pandas as pd
import phate
import math
import random
import gc
import scprep
from datetime import datetime, time
from matplotlib.animation import ImageMagickWriter
import matplotlib.animation as animation
import zipfile
from urllib.request import urlopen
import scipy.stats as st
from scipy.stats import norm
from scipy.stats import gaussian_kde
from scipy.stats import kde
from scipy.stats import binned_statistic
from scipy.stats import f_oneway
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable
plt.rcParams['pdf.fonttype'] = 42
print(sns.__version__)
from anndata import AnnData
import scanpy as sc
from delve import *
import anndata as ad
from sklearn.preprocessing import MinMaxScaler
from kh import sketch
from sklearn.cluster import KMeans
import umap
print(sc.__version__)
today = datetime.now().strftime("%m%d%Y-%H%M")

0.11.2
1.9.1


In [2]:
#Read back in the subsampled adata file
adata_save_path = r'Z:\Garrett\4i_data\112723_RPE_31day_Etop\experiment\cell_data\standard_trimmed_noPSTAT5_adata_sub.h5ad'
standard_trimmed_noPSTAT5_adata_sub = anndata.read_h5ad(adata_save_path)

c:\Users\gases\.conda\envs\python_3_7\lib\site-packages\anndata\_core\anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [3]:
# Adjusted list comprehension to exclude var_names containing "total"
columns_to_keep = [name for name in standard_trimmed_noPSTAT5_adata_sub.var_names if "total_nuc_protein" not in name]

# Selecting the data with only the columns to keep
standard_trimmed_noPSTAT5_noTotal_adata_sub = standard_trimmed_noPSTAT5_adata_sub[:, columns_to_keep]


In [4]:
def laplacian_score_fs(adata = None,
                    k: int  = None,
                    n_jobs: int  = -1):

    X, feature_names, obs_names = parse_input(adata)
    W = construct_affinity(X = X, k = k, n_jobs = n_jobs)
    scores = laplacian_score(X = X, W = W)
    predicted_features = pd.DataFrame(scores, index = feature_names, columns = ['laplacian_score'])
    predicted_features = predicted_features.sort_values(by = 'laplacian_score', ascending = True)

    return predicted_features 

In [14]:
#l_score_fullest = laplacian_score_fs(adata_fullest_sub, k = 200)
l_score_standard = laplacian_score_fs(standard_trimmed_noPSTAT5_adata_sub, k = 100)
#l_score_normalized = laplacian_score_fs(normalized_trimmed_noPSTAT5_adata_sub, k = 200)

In [15]:
len(l_score_standard)

84

In [7]:
l_score_standard.index[:46]

Index(['R5_p38_nuc_mean', 'R0_pRb_nuc_mean', 'R6_pSTAT3_ring_mean',
       'R2_IL6_ring_mean', 'R0_pRb_ratio', 'R6_p65_ring_mean',
       'R6_pSTAT3_nuc_mean', 'R5_p38_ring_mean', 'R6_p65_nuc_mean',
       'R4_IL8_ring_mean', 'R0_Rb_ratio', 'R1_p21_nuc_mean', 'R2_IL6_nuc_mean',
       'R7_GATA4_nuc_mean', 'R1_p21_ratio', 'R0_Rb_nuc_mean',
       'R7_GATA4_ring_mean', 'nuc_area', 'R5_p38_ratio', 'R7_pp38_nuc_mean',
       'R2_pp65_ring_mean', 'R1_p16_nuc_mean', 'R3_p53_ring_mean',
       'R2_pp65_nuc_mean', 'R3_PLAUR_ring_mean', 'R3_PLAUR_nuc_mean',
       'R1_Jak2_nuc_mean', 'R7_pp38_ring_mean', 'R5_STAT5_ratio',
       'R5_STAT5_nuc_mean', 'R6_p65_ratio', 'R7_pp38_ratio',
       'R1_p21_ring_mean', 'R1_p16_ring_mean', 'R4_PARP1_nuc_mean',
       'R6_pSTAT3_ratio', 'minor_axis_length', 'R5_STAT5_ring_mean',
       'R3_p53_nuc_mean', 'R1_Jak2_ratio', 'R2_IL6_ratio', 'R1_Jak2_ring_mean',
       'R2_pp65_ratio', 'R4_PARP1_ratio', 'R7_GATA4_ratio',
       'major_axis_length'],
      dtype=

For plotting 3D PHATE structures

In [ ]:
import os
import numpy as np
import phate
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as mcolors

# Define parameter ranges
count_list = [200]
index_ranges = [46]
t_values = ['auto']
gamma_values = [1]

# Ensure the directory for saving plots exists
plots_dir = 'phate_plots'
os.makedirs(plots_dir, exist_ok=True)

# Loop over each combination of parameters
for count in count_list:
    for index_range in index_ranges:
        for t in t_values:
            for gamma in gamma_values:
                # Initialize PHATE operator with varying t and gamma, projecting to three dimensions
                phate_op = phate.PHATE(knn=count, t=t, gamma=gamma, n_components=3)

                # Fit-transform PHATE
                X_phate = phate_op.fit_transform(standard_trimmed_noPSTAT5_adata_sub.X[:, np.isin(standard_trimmed_noPSTAT5_adata_sub.var_names, l_score_standard.index[:index_range])])
                
                # Set PHATE result for plotting
                standard_trimmed_noPSTAT5_adata_sub.obsm['X_phate'] = X_phate
                
                # Generate plot title
                plot_title = f'PHATE: Neighbors={count}, Index Range={index_range}, t={t}, gamma={gamma}'

                # Define angles for rotation in degrees
                angles = [(30, 30), (30, 120), (30, 210), (30, 300)]

                # Define unique labels and their corresponding colors
                labels = np.unique(standard_trimmed_noPSTAT5_adata_sub.obs['sample_ID'])
                color_map = plt.get_cmap('viridis', len(labels))
                colors = color_map(np.linspace(0, 1, len(labels)))
                color_dict = dict(zip(labels, colors))

                for i, (elev, azim) in enumerate(angles, start=1):
                    fig = plt.figure()
                    ax = fig.add_subplot(111, projection='3d')
                    # Plot each label with a unique color
                    for label, color in color_dict.items():
                        idx = standard_trimmed_noPSTAT5_adata_sub.obs['sample_ID'] == label
                        ax.scatter(X_phate[idx, 0], X_phate[idx, 1], X_phate[idx, 2], color=color, label=label, s=5)
                    
                    ax.view_init(elev=elev, azim=azim)
                    ax.set_title(plot_title)
                    ax.legend(title='Sample ID', bbox_to_anchor=(1.05, 1), loc='upper left')
                    plt.savefig(f'{plots_dir}/plot_neighbors_{count}_indexrange_{index_range}_t_{t}_gamma_{gamma}_3d_angle_{i}.png', bbox_inches='tight')
                    plt.close()

print("3D PHATE plots saved with multiple angles and legends.")


After you've decided on a final structure, you can save the ADATA file. It should have the PHATE coordinates saved as part of it now, removing the need to recompute the phate structure if you want to plot different things onto it later

In [ ]:
#Save the entire adata file with new PHATE embeddings
adata_save_path = r'my\save\path\standard_adata_sub_sub.h5ad'
standard_trimmed_noPSTAT5_adata_sub.write_h5ad(adata_save_path)